In [1]:
import pandas as pd
import sqlite3

## 1) Load Excel data tabs into dataframes

In [2]:
sales = pd.read_excel(
    'data/US_Regional_Sales_Data.xlsx', 
    sheet_name='sales',
    header=0)

customers = pd.read_excel(
    'data/US_Regional_Sales_Data.xlsx', 
    sheet_name='customers',
    header=0)

store_locations = pd.read_excel(
    'data/US_Regional_Sales_Data.xlsx', 
    sheet_name='store_locations',
    header=0)

products = pd.read_excel(
    'data/US_Regional_Sales_Data.xlsx', 
    sheet_name='products',
    header=0)

regions = pd.read_excel(
    'data/US_Regional_Sales_Data.xlsx', 
    sheet_name='regions',
    header=0)

sales_team = pd.read_excel(
    'data/US_Regional_Sales_Data.xlsx', 
    sheet_name='sales_team',
    header=0)


In [3]:
sales.head(2)

,OrderNumber,SalesChannel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,SalesTeamID,CustomerID,StoreID,ProductID,OrderQuantity,DiscountApplied,UnitPrice,UnitCost
0,SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,259,12,5,0.075,1963.1,1001.181
1,SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,196,27,3,0.075,3939.6,3348.660


In [4]:
store_locations.head(2)

,StoreID,CityName,County,StateCode,State,Type,Latitude,Longitude,AreaCode,Population,HouseholdIncome,MedianIncome,LandArea,WaterArea,TimeZone
0,1,Birmingham,Shelby County/Jefferson County,AL,Alabama,City,33.52744,-86.79905,205,212461,89972,31061,378353942,6591013,America/Chicago
1,2,Huntsville,Limestone County/Madison County,AL,Alabama,City,34.69901,-86.67298,256,190582,78554,48775,552604579,3452021,America/Chicago


## 2) Create sqllite database

In [14]:
db_conn = sqlite3.connect("data/us_sales.db")

In [20]:
c = db_conn.cursor()

## 3) Create sqllite tables

In [69]:
# store_locations
c.execute(
    """
    
CREATE TABLE store_locations (
    StoreID INTEGER,
    CityName TEXT NOT NULL,
    County TEXT NOT NULL,
    StateCode TEXT NOT NULL,
    State TEXT NOT NULL,
    Type TEXT NOT NULL,
    Latitude REAL,
    Longitude REAL,
    AreaCode INTEGER,
    Population INTEGER,
    HouseholdIncome INTEGER,
    MedianIncome INTEGER,
    LandArea INTEGER,
    WaterArea INTEGER,
    TimeZone TEXT NOT NULL,
    PRIMARY KEY (StoreID)
        );
     """
)

In [67]:
# sales_team
c.execute(
    """
    
CREATE TABLE sales_team (
    SalesTeamID INTEGER,
    SalesTeam TEXT NOT NULL,
    Region TEXT NOT NULL,
    PRIMARY KEY (SalesTeamID)
        );
     """
)

In [65]:
# regions
c.execute(
    """
    
CREATE TABLE regions (
    StateCode TEXT NOT NULL,
    State TEXT NOT NULL,
    Region TEXT NOT NULL,
    PRIMARY KEY (StateCode)
        );
     """
)

In [63]:
# products
c.execute(
    """
    
CREATE TABLE products (
    ProductID INTEGER,
    ProductName TEXT NOT NULL,
    PRIMARY KEY (ProductID)
        );
     """
)

In [40]:
# customers
c.execute(
    """
    
CREATE TABLE customers (
    CustomerId INTEGER,
    CustomerNames TEXT NOT NULL,
    PRIMARY KEY (CustomerID)
        );
     """
)

c.execute(
    """

DROP TABLE sales
    
    """
)

In [32]:
# SALES
c.execute(
    """

CREATE TABLE sales (
    OrderNumber TEXT NOT NULL,
    SalesChannel TEXT NOT NULL,
    WarehouseCode TEXT NOT NULL,
    ProcuredDate TEXT NOT NULL,
    OrderDate TEXT NOT NULL,
    ShipDate TEXT NOT NULL,
    DeliveryDate TEXT NOT NULL,
    CurrencyCode TEXT NOT NULL,
    SalesTeamID INTEGER,
    CustomerID, INTEGER,
    StoreID INTEGER,
    ProductID INTEGER,
    OrderQuantity INTEGER,
    DiscountApplied REAL,
    UnitPrice REAL,
    UnitCost REAL,
    PRIMARY KEY (OrderNumber)
        );
     """
)

In [76]:
sales.to_sql('sales', db_conn, if_exists='append', index=False)
customers.to_sql('customers', db_conn, if_exists='append', index=False)
store_locations.to_sql('store_locations', db_conn, if_exists='append', index=False)
products.to_sql('products', db_conn, if_exists='append', index=False)
regions.to_sql('regions', db_conn, if_exists='append', index=False)
sales_team.to_sql('sales_team', db_conn, if_exists='append', index=False)

In [48]:
pd.read_sql("SELECT * FROM customers c join sales s on c.customerid = s.customerid LIMIT 5 ",  db_conn)

,CustomerId,CustomerNames,OrderNumber,SalesChannel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,SalesTeamID,CustomerID,INTEGER,StoreID,ProductID,OrderQuantity,DiscountApplied,UnitPrice,UnitCost
0,15,Rochester Ltd,SO - 000101,In-Store,WARE-UHY1004,2017-12-31 00:00:00,2018-05-31 00:00:00,2018-06-14 00:00:00,2018-06-19 00:00:00,USD,6,15,None,259,12,5,0.075,1963.1,1001.181
1,20,Pacific Ltd,SO - 000102,Online,WARE-NMK1003,2017-12-31 00:00:00,2018-05-31 00:00:00,2018-06-22 00:00:00,2018-07-02 00:00:00,USD,14,20,None,196,27,3,0.075,3939.6,3348.660
2,16,"3LAB, Ltd",SO - 000103,Distributor,WARE-UHY1004,2017-12-31 00:00:00,2018-05-31 00:00:00,2018-06-21 00:00:00,2018-07-01 00:00:00,USD,21,16,None,213,16,1,0.050,1775.5,781.220
3,48,"Fenwal, Corp",SO - 000104,Wholesale,WARE-NMK1003,2017-12-31 00:00:00,2018-05-31 00:00:00,2018-06-02 00:00:00,2018-06-07 00:00:00,USD,28,48,None,107,23,8,0.075,2324.9,1464.687
4,49,Bare,SO - 000105,Distributor,WARE-NMK1003,2018-04-10 00:00:00,2018-05-31 00:00:00,2018-06-16 00:00:00,2018-06-26 00:00:00,USD,22,49,None,111,26,8,0.100,1822.4,1476.144


In [77]:
pd.read_sql("SELECT * FROM regions",  db_conn)

,StateCode,State,Region
0,AL,Alabama,South
1,AR,Arkansas,South
2,AZ,Arizona,West
3,CA,California,West
4,CO,Colorado,West
5,CT,Connecticut,Northeast
6,DC,District of Columbia,South
7,DE,Delaware,South
8,FL,Florida,South
9,GA,Georgia,South


In [57]:
df = pd.read_sql_query(
con = db_conn,
    sql = """select *
             from sales s
             join customers c
             on s.customerid = c.customerid
    
    """

)

In [58]:
df.head(3)

,OrderNumber,SalesChannel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,SalesTeamID,CustomerID,INTEGER,StoreID,ProductID,OrderQuantity,DiscountApplied,UnitPrice,UnitCost,CustomerId,CustomerNames
0,SO - 000101,In-Store,WARE-UHY1004,2017-12-31 00:00:00,2018-05-31 00:00:00,2018-06-14 00:00:00,2018-06-19 00:00:00,USD,6,15,None,259,12,5,0.075,1963.1,1001.181,15,Rochester Ltd
1,SO - 000102,Online,WARE-NMK1003,2017-12-31 00:00:00,2018-05-31 00:00:00,2018-06-22 00:00:00,2018-07-02 00:00:00,USD,14,20,None,196,27,3,0.075,3939.6,3348.660,20,Pacific Ltd
2,SO - 000103,Distributor,WARE-UHY1004,2017-12-31 00:00:00,2018-05-31 00:00:00,2018-06-21 00:00:00,2018-07-01 00:00:00,USD,21,16,None,213,16,1,0.050,1775.5,781.220,16,"3LAB, Ltd"


In [ ]:
db_conn.close()

In [ ]:
db_conn = sqlite3.connect("data/us_sales.db")
## don't need cursor if you're not writing to table